In [5]:
!pip install scipy

## Item to item recommendations

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import sparse
from sklearn.preprocessing import normalize

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
interactions_df = pd.read_csv('/kaggle/input/lastfm-music-artist-scrobbles/lastfm_user_scrobbles.csv')
titles_df = pd.read_csv('/kaggle/input/lastfm-music-artist-scrobbles/lastfm_artist_list.csv')

In [8]:
interactions_df.tail()

In [9]:
titles_df

Что внутри???

In [10]:
interactions_df.groupby('user_id', as_index=False).count()

In [11]:
### Посмотрим на распределение артистов по пользователям

interactions_df.groupby('user_id', as_index=False).count().mean()

In [12]:
### Artist's names

titles_df.index = titles_df['artist_id']
title_dict = titles_df['artist_name'].to_dict()

In [13]:
### Make user_item matrix

rows, r_pos = np.unique(interactions_df.values[:, 0], return_inverse=True)

# return_inverse - True или False (необязательный)
# Если True то помимо самих уникальных элементов так же будут 
# возвращаться индексы уникального массива, которые можно 
# использовать для восстановления входного массива.

In [14]:
cols, c_pos = np.unique(interactions_df.values[:, 1], return_inverse=True)

In [20]:
# Sparse matrix
# Compressed Sparse Row matrix

interactions_sparse = sparse.csr_matrix((interactions_df.values[:, 2], (r_pos, c_pos)))

In [21]:
Pui = normalize(interactions_sparse, norm='l2', axis=1)
sim = Pui.T * Pui # косинусное расстояние

In [24]:
sim.todense()

In [27]:
# Eligability check (top similarity)
group_ind = 10734

[title_dict[i+1] for i in sim[group_ind].toarray().argsort()[0][-20:]]

## Item to user

In [34]:
interactions_sparse_transposed = interactions_sparse.transpose(copy=True)
Pui = normalize(interactions_sparse_transposed, norm='l2', axis=1)

fit = Pui * Pui * Pui

In [35]:
# What we start 

init_set = set([title_dict[i+1] for i in fit[520].toarray.argsort()[0][-70:].tolist()])

In [36]:
# Our recommendations to user 

pred_set - init_set